# INVALID_TOOL_RESULTS

You are passing too many, too few, or mismatched [`ToolMessages`](https://api.js.langchain.com/classes/_langchain_core.messages_tool.ToolMessage.html) to a model.

When [using a model to call tools](/docs/concepts#functiontool-calling), the [`AIMessage`](https://api.js.langchain.com/classes/_langchain_core.messages.AIMessage.html)
the model responds with will contain a `tool_calls` array. To continue the flow, the next messages you pass back to the model must
be exactly one `ToolMessage` for each item in that array containing the result of that tool call. Each `ToolMessage` must have a `tool_call_id` field
that matches one of the `tool_calls` on the `AIMessage`.

For example, given the following response from a model:

In [1]:
import { z } from "zod";
import { tool } from "@langchain/core/tools";
import { ChatOpenAI } from "@langchain/openai";
import { BaseMessageLike } from "@langchain/core/messages";

const model = new ChatOpenAI({
  model: "gpt-4o-mini",
});

const dummyTool = tool(
  async () => {
    return "action complete!";
  },
  {
    name: "foo",
    schema: z.object({}),
  }
);

const modelWithTools = model.bindTools([dummyTool]);

const chatHistory: BaseMessageLike[] = [
  {
    role: "user",
    content: `Call tool "foo" twice with no arguments`,
  },
];

const responseMessage = await modelWithTools.invoke(chatHistory);

console.log(responseMessage);

AIMessage {
  "id": "msg_01S9PN4T1RPvn1tHWJysBVt3",
  "content": [
    {
      "type": "text",
      "text": "Certainly! I'll call the \"foo\" tool twice with no arguments as you've requested. The \"foo\" tool doesn't require any parameters, so we can proceed with the calls directly."
    },
    {
      "type": "tool_use",
      "id": "toolu_01EN2nxE8M5gr7rJ3DvEUjoT",
      "name": "foo",
      "input": {}
    },
    {
      "type": "tool_use",
      "id": "toolu_01T2Fp8zgtWtWBacoT1LYn75",
      "name": "foo",
      "input": {}
    }
  ],
  "additional_kwargs": {
    "id": "msg_01S9PN4T1RPvn1tHWJysBVt3",
    "type": "message",
    "role": "assistant",
    "model": "claude-3-5-sonnet-20240620",
    "stop_reason": "tool_use",
    "stop_sequence": null,
    "usage": {
      "input_tokens": 371,
      "output_tokens": 92
    }
  },
  "response_metadata": {
    "id": "msg_01S9PN4T1RPvn1tHWJysBVt3",
    "model": "claude-3-5-sonnet-20240620",
    "stop_reason": "tool_use",
    "stop_sequence"

Calling the model with only one tool response would result in an error:

In [2]:
const toolResponse1 = await dummyTool.invoke(responseMessage.tool_calls![0]);

chatHistory.push(responseMessage);
chatHistory.push(toolResponse1);

await modelWithTools.invoke(chatHistory);

BadRequestError: 400 {"type":"error","error":{"type":"invalid_request_error","message":"messages.2: the following `tool_use` ids were not found in `tool_result` blocks: {'toolu_01T2Fp8zgtWtWBacoT1LYn75'}"}}
    at APIError.generate (/Users/jacoblee/langchain/langchainjs/node_modules/@anthropic-ai/sdk/error.js:41:20)
    at Anthropic.makeStatusError (/Users/jacoblee/langchain/langchainjs/node_modules/@anthropic-ai/sdk/core.js:275:33)
    at Anthropic.makeRequest (/Users/jacoblee/langchain/langchainjs/node_modules/@anthropic-ai/sdk/core.js:318:30)
    at process.processTicksAndRejections (node:internal/process/task_queues:95:5)
    at async RetryOperation._fn (/Users/jacoblee/langchain/langchainjs/node_modules/p-retry/index.js:50:12) {
  status: 400,
  headers: {
    'cf-cache-status': 'DYNAMIC',
    'cf-ray': '8d31c6654b9132c5-PHL',
    connection: 'keep-alive',
    'content-length': '185',
    'content-type': 'application/json',
    date: 'Tue, 15 Oct 2024 18:12:01 GMT',
    'request-i

If we add a second response, the call will succeed as expected because we now have one tool response per tool call:

In [3]:
const toolResponse2 = await dummyTool.invoke(responseMessage.tool_calls![1]);

chatHistory.push(toolResponse2);

await modelWithTools.invoke(chatHistory);

AIMessage {
  "id": "msg_0136ZL6zZEQarNmiFrBVTizV",
  "content": "I've called the \"foo\" tool twice as requested. Both calls were successful, and each returned the output \"action complete!\". Is there anything else you'd like me to do with the \"foo\" tool or any other questions you have?",
  "additional_kwargs": {
    "id": "msg_0136ZL6zZEQarNmiFrBVTizV",
    "type": "message",
    "role": "assistant",
    "model": "claude-3-5-sonnet-20240620",
    "stop_reason": "end_turn",
    "stop_sequence": null,
    "usage": {
      "input_tokens": 525,
      "output_tokens": 54
    }
  },
  "response_metadata": {
    "id": "msg_0136ZL6zZEQarNmiFrBVTizV",
    "model": "claude-3-5-sonnet-20240620",
    "stop_reason": "end_turn",
    "stop_sequence": null,
    "usage": {
      "input_tokens": 525,
      "output_tokens": 54
    },
    "type": "message",
    "role": "assistant"
  },
  "tool_calls": [],
  "invalid_tool_calls": [],
  "usage_metadata": {
    "input_tokens": 525,
    "output_tokens": 

But if we add a duplicate, extra tool response, the call will fail again:

In [4]:
const duplicateToolResponse2 = await dummyTool.invoke(responseMessage.tool_calls![1]);

chatHistory.push(duplicateToolResponse2);

await modelWithTools.invoke(chatHistory);

BadRequestError: 400 {"type":"error","error":{"type":"invalid_request_error","message":"messages.2: Too many `tool_result` blocks found: 3, expected 2. The number of `tool_result` blocks must match the number of `tool_use` blocks in the previous message."}}
    at APIError.generate (/Users/jacoblee/langchain/langchainjs/node_modules/@anthropic-ai/sdk/error.js:41:20)
    at Anthropic.makeStatusError (/Users/jacoblee/langchain/langchainjs/node_modules/@anthropic-ai/sdk/core.js:275:33)
    at Anthropic.makeRequest (/Users/jacoblee/langchain/langchainjs/node_modules/@anthropic-ai/sdk/core.js:318:30)
    at process.processTicksAndRejections (node:internal/process/task_queues:95:5)
    at async RetryOperation._fn (/Users/jacoblee/langchain/langchainjs/node_modules/p-retry/index.js:50:12) {
  status: 400,
  headers: {
    'cf-cache-status': 'DYNAMIC',
    'cf-ray': '8d31c6bc89aa32c6-PHL',
    connection: 'keep-alive',
    'content-length': '236',
    'content-type': 'application/json',
    da

You should additionally not pass `ToolMessages` back to to a model if they are not preceded by an `AIMessage` with tool calls. For example, this will fail:

In [5]:
await modelWithTools.invoke([{
  role: "tool",
  content: "action completed!",
  tool_call_id: "dummy",
}])

BadRequestError: 400 {"type":"error","error":{"type":"invalid_request_error","message":"messages.0: `tool_result` block(s) provided when previous message does not contain any `tool_use` blocks"}}
    at APIError.generate (/Users/jacoblee/langchain/langchainjs/node_modules/@anthropic-ai/sdk/error.js:41:20)
    at Anthropic.makeStatusError (/Users/jacoblee/langchain/langchainjs/node_modules/@anthropic-ai/sdk/core.js:275:33)
    at Anthropic.makeRequest (/Users/jacoblee/langchain/langchainjs/node_modules/@anthropic-ai/sdk/core.js:318:30)
    at process.processTicksAndRejections (node:internal/process/task_queues:95:5)
    at async RetryOperation._fn (/Users/jacoblee/langchain/langchainjs/node_modules/p-retry/index.js:50:12) {
  status: 400,
  headers: {
    'cf-cache-status': 'DYNAMIC',
    'cf-ray': '8d31ce546dbc4cb4-PHL',
    connection: 'keep-alive',
    'content-length': '174',
    'content-type': 'application/json',
    date: 'Tue, 15 Oct 2024 18:17:26 GMT',
    'request-id': 'req_01

See [this guide](/docs/how_to/tool_results_pass_to_model/) for more details on tool calling.

## Troubleshooting

The following may help resolve this error:

- If you are using a custom executor rather than a prebuilt one like LangGraph's [`ToolNode`](https://langchain-ai.github.io/langgraphjs/reference/classes/langgraph_prebuilt.ToolNode.html)
  or the legacy LangChain [AgentExecutor](/docs/how_to/agent_executor), verify that you are invoking and returning the result for one tool per tool call.
- If you are using [few-shot tool call examples](/docs/how_to/tools_few_shot) with messages that you manually create, and you want to simulate a failure,
  you still need to pass back a `ToolMessage` whose content indicates that failure.
